Mounting the drive

In [2]:
#mounting the drive 
from google.colab import drive
drive.mount('/content/drive')
print("done")

Mounted at /content/drive
done


In [1]:
path='C:\\Users\\Janidu Chathumina\\Desktop\\Dataset\\'

In [2]:
!nvidia-smi

Thu Mar 31 16:29:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 497.17       Driver Version: 497.17       CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   55C    P8     7W /  N/A |    134MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Imports

In [3]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from keras import backend as K
from keras.layers import Layer,InputSpec
import keras.layers as kl
from glob import glob
from sklearn.metrics import auc
from keras.preprocessing import image
from tensorflow.keras.models import Sequential
from sklearn.metrics import roc_auc_score
from tensorflow.keras import callbacks 
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras import Model
from tensorflow.keras.layers import concatenate,Dense, Conv2D, MaxPooling2D, Flatten,Input,Activation,add,BatchNormalization,Dropout

import shutil
from sklearn.metrics import  precision_score, recall_score, accuracy_score,classification_report
from tensorflow.python.platform import build_info as tf_build_info
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

#Data Preprocessing

##Initializing the dataset

In [4]:
data_pd = pd.read_csv(path+'HAM10000_metadata')
data_pd.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern


In [5]:
#counting the images per each class
def count_images_on_type(series):
    display(pd.DataFrame(series.value_counts()))

#class_list = ['akiec','bcc','bkl','df','mel','nv','vasc']
count_images_on_type(data_pd['dx'])


,dx
nv,6705
mel,1113
bkl,1099
bcc,514
akiec,327
vasc,142
df,115


In [6]:
train_dir = os.path.join('HAM10000', 'train_dir')

test_dir = os.path.join('HAM10000', 'test_dir')


In [ ]:

#making the directories to store test and train data
os.mkdir('/content/drive/MyDrive/Dataset/train_dir')
os.mkdir('/content/drive/MyDrive/Dataset/test_dir')

In [7]:
df_count = data_pd.groupby('lesion_id').count()
df_count.head()

,image_id,dx,dx_type,age,sex,localization,dataset
lesion_id,,,,,,,
HAM_0000000,2,2,2,2,2,2,2
HAM_0000001,1,1,1,1,1,1,1
HAM_0000002,3,3,3,3,3,3,3
HAM_0000003,1,1,1,1,1,1,1
HAM_0000004,1,1,1,1,1,1,1


In [8]:
df_count = df_count[df_count['dx'] == 1]
df_count.reset_index(inplace=True)
df_count.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset
0,HAM_0000001,1,1,1,1,1,1,1
1,HAM_0000003,1,1,1,1,1,1,1
2,HAM_0000004,1,1,1,1,1,1,1
3,HAM_0000007,1,1,1,1,1,1,1
4,HAM_0000008,1,1,1,1,1,1,1


In [9]:
#finding the duplicates to make sure none of these images have augmented duplicates

def duplicates(x):
    unique = set(df_count['lesion_id'])
    if x in unique:
        return 'no' 
    else:
        return 'duplicate'

data_pd['is_duplicate'] = data_pd['lesion_id'].apply(duplicates)
data_pd.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,is_duplicate
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern,duplicate
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern,duplicate
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern,duplicate
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern,duplicate
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern,duplicate


In [10]:
#framing all the meta data
df_count = data_pd[data_pd['is_duplicate'] == 'no']
print(df_count)

         lesion_id      image_id     dx dx_type   age     sex localization  \
10     HAM_0001396  ISIC_0025276    bkl   histo  55.0  female        trunk   
15     HAM_0007207  ISIC_0031326    bkl   histo  65.0    male         back   
20     HAM_0006071  ISIC_0032343    bkl   histo  70.0  female         face   
33     HAM_0005612  ISIC_0024981    bkl   histo  80.0    male        scalp   
34     HAM_0005388  ISIC_0027815    bkl   histo  80.0    male        chest   
...            ...           ...    ...     ...   ...     ...          ...   
9988   HAM_0001036  ISIC_0027588  akiec   histo  50.0  female         face   
9990   HAM_0004462  ISIC_0027334  akiec   histo  45.0    male        trunk   
9991   HAM_0001152  ISIC_0030133  akiec   histo  65.0    male         face   
10001  HAM_0000020  ISIC_0031922  akiec   histo  60.0  female         face   
10008  HAM_0001576  ISIC_0033705  akiec   histo  60.0    male         face   

            dataset is_duplicate  
10     vidir_modern         

##Test Train Split 15% to 85%

In [11]:
train, test_df = train_test_split(df_count, test_size=0.15, stratify=df_count['dx'])

In [12]:
#labels the data as test or train
def identify_trainOrtest(x):
    test_data = set(test_df['image_id'])
    if str(x) in test_data:
        return 'test'
    else:
        return 'train'


In [13]:
test_df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,is_duplicate
9444,HAM_0000488,ISIC_0032930,nv,consensus,40.0,male,back,vidir_modern,no
8044,HAM_0002606,ISIC_0033228,nv,consensus,25.0,male,unknown,vidir_modern,no
4326,HAM_0001127,ISIC_0029307,nv,follow_up,50.0,female,trunk,vidir_molemax,no
1004,HAM_0007151,ISIC_0029793,bkl,consensus,70.0,female,trunk,vidir_molemax,no
3348,HAM_0000812,ISIC_0032442,nv,follow_up,50.0,female,trunk,vidir_molemax,no


In [14]:
#creating train_df
data_pd['train_test_split'] = data_pd['image_id'].apply(identify_trainOrtest)
train_df = data_pd[data_pd['train_test_split'] == 'train']
train_df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,is_duplicate,train_test_split
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern,duplicate,train
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern,duplicate,train
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern,duplicate,train
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern,duplicate,train
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern,duplicate,train


In [15]:
# adding to lists by image id of train and test images
train_list = list(train_df['image_id'])
test_list = list(test_df['image_id'])

In [16]:
len(train_list)

9187

In [17]:
len(test_list)

828

In [18]:
# Set the image_id as the index in data_pd
data_pd.set_index('image_id', inplace=True)

In [19]:
targetnames = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

In [33]:
#making the directories per each cancer type in test and train dir 
for i in targetnames:
  os.mkdir(path+"test_dir//"+i)
  os.mkdir(path+"train_dir//"+i)

###Copying the images to the test and the train folders

In [34]:
#compying images to train folders
for image in train_list:
    file_name = image+'.jpg'
    label = data_pd.loc[image, 'dx']

    # path of source image 
    source = os.path.join(path+'HAM10000//',file_name)

    # copying the image from the source to target file
    target = os.path.join(path+'train_dir//',label,file_name) 

    shutil.copyfile(source, target)

In [35]:
#copying images to the test folders
for image in test_list:

    file_name = image+'.jpg'
    label = data_pd.loc[image, 'dx']

    # path of source image 
    source = os.path.join(path+'HAM10000//',file_name)

    # copying the image from the source to target file
    target = os.path.join(path+'test_dir//',label,file_name)

    shutil.copyfile(source, target)

##Image Augmentation

In [36]:
targetnames = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

# Augmenting images and storing them in temporary directories 
for img_class in targetnames:

    #creating temporary directories
    # creating a base directory
    
    os.mkdir(path+'aug_dir')
    # creating a subdirectory inside the base directory for images of the same class
    
    os.mkdir(path+'aug_dir//img_dir')

    img_list = os.listdir(path+'train_dir//' + img_class)

    # Copy images from the class train_dir to the img_dir 
    for file_name in img_list:

        # path of source image in training directory
        source = os.path.join(path+'train_dir//',img_class, file_name)

        # creating a target directory to send images 
        target = os.path.join(path+'aug_dir//img_dir//',file_name)

        # copying the image from the source to target file
        shutil.copyfile(source, target)

    # Temporary augumented dataset directory.
    source_path = path+'aug_dir//'

    # Augmented images will be saved to training directory
    save_path = path+'train_dir//'+img_class

    # Creating Image Data Generator to augment images
    # Rotating,flipping and shifting the images to create augmented images.
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(

        rotation_range=180,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest'

    )

    batch_size = 50

    aug_datagen = datagen.flow_from_directory(source_path,
                                              save_to_dir=save_path,
                                              save_format='jpg',
                                              target_size=(224, 224),
                                              batch_size=batch_size)

    # Generate the augmented images
    aug_images = 8000 

    num_files = len(os.listdir(path+'aug_dir//img_dir//'))
    num_batches = int(np.ceil((aug_images - num_files) / batch_size))

    # creating 8000 augmented images per class
    for i in range(0, num_batches):
        images, labels = next(aug_datagen)

    # delete temporary directory 
    shutil.rmtree(path+'aug_dir')


Found 304 images belonging to 1 classes.
Found 488 images belonging to 1 classes.
Found 1033 images belonging to 1 classes.
Found 109 images belonging to 1 classes.
Found 1079 images belonging to 1 classes.
Found 6042 images belonging to 1 classes.
Found 132 images belonging to 1 classes.


In [22]:
train_path = path+'train_dir'
test_path = path+'test_dir'
batch_size = 32

In [23]:

datagen=ImageDataGenerator()

In [24]:
#To find the total number of images in the training batch and test batch.
image_size = 299
print("\nTrain Batches: ")
train_batches = datagen.flow_from_directory(directory=train_path,
                                            target_size=(image_size,image_size),
                                            batch_size=batch_size,
                                            shuffle=True)

print("\nTest Batches: ")
test_batches =datagen.flow_from_directory(test_path,
                                           target_size=(image_size,image_size),
                                           batch_size=batch_size,
                                           shuffle=False)


Train Batches: 
Found 51699 images belonging to 7 classes.

Test Batches: 
Found 828 images belonging to 7 classes.


In [25]:
#image shape width =299 height=299 (depth=3) RGB
train_batches.image_shape

(299, 299, 3)

In [26]:
print(train_batches.filenames)

['akiec\\ISIC_0024329.jpg', 'akiec\\ISIC_0024372.jpg', 'akiec\\ISIC_0024418.jpg', 'akiec\\ISIC_0024450.jpg', 'akiec\\ISIC_0024463.jpg', 'akiec\\ISIC_0024468.jpg', 'akiec\\ISIC_0024470.jpg', 'akiec\\ISIC_0024511.jpg', 'akiec\\ISIC_0024517.jpg', 'akiec\\ISIC_0024522.jpg', 'akiec\\ISIC_0024539.jpg', 'akiec\\ISIC_0024562.jpg', 'akiec\\ISIC_0024575.jpg', 'akiec\\ISIC_0024579.jpg', 'akiec\\ISIC_0024646.jpg', 'akiec\\ISIC_0024654.jpg', 'akiec\\ISIC_0024707.jpg', 'akiec\\ISIC_0024710.jpg', 'akiec\\ISIC_0024763.jpg', 'akiec\\ISIC_0024771.jpg', 'akiec\\ISIC_0024800.jpg', 'akiec\\ISIC_0024843.jpg', 'akiec\\ISIC_0024923.jpg', 'akiec\\ISIC_0024946.jpg', 'akiec\\ISIC_0024948.jpg', 'akiec\\ISIC_0025029.jpg', 'akiec\\ISIC_0025069.jpg', 'akiec\\ISIC_0025089.jpg', 'akiec\\ISIC_0025178.jpg', 'akiec\\ISIC_0025182.jpg', 'akiec\\ISIC_0025196.jpg', 'akiec\\ISIC_0025247.jpg', 'akiec\\ISIC_0025264.jpg', 'akiec\\ISIC_0025319.jpg', 'akiec\\ISIC_0025350.jpg', 'akiec\\ISIC_0025358.jpg', 'akiec\\ISIC_0025368.jpg', 

#Model

In [41]:
#Model Parameters
input_shape = (299, 299, 3)
num_classes = 7

#initialize a sequential model
model = Sequential()

#conv2D(num_filters, kernal(filter)_size, activation, padding, input_shape)


model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',padding = 'Same',input_shape=input_shape))
model.add(BatchNormalization())

################
model.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.10))

model.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
model.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
model.add(BatchNormalization())

model.add(Conv2D(128, (5, 5), activation='relu',padding = 'Same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.10))
################
model.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.10))

model.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
model.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
model.add(BatchNormalization())

model.add(Conv2D(128, (5, 5), activation='relu',padding = 'Same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.10))
################

#model.add(BatchNormalization())

#flattening layer
model.add(Flatten())


model.add(Activation('relu'))
model.add(Dropout(0.10))

#7x7=49 all possibilities 
model.add(Dense(49, activation='relu'))
model.add(BatchNormalization())

#Output

model.add(Dense(num_classes, activation='softmax'))

#Model Summary

In [42]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 299, 299, 32)      896       
_________________________________________________________________
batch_normalization_8 (Batch (None, 299, 299, 32)      128       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 299, 299, 64)      18496     
_________________________________________________________________
batch_normalization_9 (Batch (None, 299, 299, 64)      256       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 149, 149, 64)      0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 149, 149, 64)      0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 149, 149, 64)     

#Model Evaluation Before Training

In [29]:
batch_size=16
#taking predictions from the trained model against the test images
predictions = model.predict(test_batches, steps=len(test_df)/batch_size, verbose=0)

In [30]:
#geting predictions on test dataset
y_pred = np.argmax(predictions, axis=1)
targetnames = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
#getting the true labels per image 
y_true = test_batches.classes



# Creating classification report 
report = classification_report(y_true, y_pred, target_names=targetnames)

print("\nClassification Report BEFORE TRAINING for all 7 calsses :")
print("\n")
print(report)


Classification Report BEFORE TRAINING for all 7 calsses :


              precision    recall  f1-score   support

       akiec       0.00      0.00      0.00        23
         bcc       0.12      0.08      0.09        26
         bkl       0.00      0.00      0.00        66
          df       0.00      0.00      0.00         6
         mel       0.04      0.94      0.08        34
          nv       0.00      0.00      0.00       663
        vasc       0.00      0.00      0.00        10

    accuracy                           0.04       828
   macro avg       0.02      0.15      0.02       828
weighted avg       0.01      0.04      0.01       828



C:\Users\Janidu Chathumina\anaconda3\envs\SDGP\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Janidu Chathumina\anaconda3\envs\SDGP\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Janidu Chathumina\anaconda3\envs\SDGP\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

#Training the Model

In [43]:
opt1=tf.keras.optimizers.Adam(learning_rate=0.01,epsilon=0.1)

#compiling the model
model.compile(optimizer = opt1 , loss = "categorical_crossentropy", metrics=["accuracy"])

In [48]:
checkpoint_path =path+'Callbacks\\CNN_CP.cpkt'
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path,
    save_weights_only=True,
    verbose=1
)

Earlystop = EarlyStopping(monitor='val_loss', mode='min',patience=10, min_delta=0.001)

In [49]:
##Loading the weights of the last training session
model.load_weights(checkpoint_path)

In [45]:
class_weights = {   
                    0 : 1.0,  ## akiec
                    1 : 1.0,  ## bcc
                    2 : 1.0,  ## bkl
                    3 : 1.0,  ## df
                    4 : 5.0,  ## mel
                    5 : 1.0,  ## nv
                    6 : 1.0,  ## vasc
                }

In [50]:
batch_size=32

#training
history = model.fit(train_batches,
                    batch_size=32,
                    epochs = 20,
                    validation_data = test_batches,
                    verbose = 1,
                    validation_steps=len(test_df)/batch_size,
                    steps_per_epoch=len(train_df)/batch_size, 
                    callbacks=[cp_callback,Earlystop],
                    class_weight=class_weights)


Epoch 1/20
287/287 [==============================] - 2344s 8s/step - loss: 1.7898 - accuracy: 0.4368 - val_loss: 0.7492 - val_accuracy: 0.7645

Epoch 00001: saving model to C:\Users\Janidu Chathumina\Desktop\Dataset\Callbacks\CNN_CP.cpkt
Epoch 2/20
287/287 [==============================] - 2365s 8s/step - loss: 1.6643 - accuracy: 0.4678 - val_loss: 0.6803 - val_accuracy: 0.7560

Epoch 00002: saving model to C:\Users\Janidu Chathumina\Desktop\Dataset\Callbacks\CNN_CP.cpkt
Epoch 3/20
287/287 [==============================] - 2540s 9s/step - loss: 1.5963 - accuracy: 0.4924 - val_loss: 1.4977 - val_accuracy: 0.5157

Epoch 00003: saving model to C:\Users\Janidu Chathumina\Desktop\Dataset\Callbacks\CNN_CP.cpkt
Epoch 4/20
287/287 [==============================] - 2428s 8s/step - loss: 1.5227 - accuracy: 0.5169 - val_loss: 0.8372 - val_accuracy: 0.7343

Epoch 00004: saving model to C:\Users\Janidu Chathumina\Desktop\Dataset\Callbacks\CNN_CP.cpkt
Epoch 5/20
287/287 [========================

In [53]:
model.save(path+'CNN_Model_2.h5')

#Model Evaluation after Training

In [51]:
batch_size=16
#taking predictions from the trained model against the test images
predictions = model.predict(test_batches, steps=len(test_df)/batch_size, verbose=0)


In [52]:
#geting predictions on test dataset
y_pred = np.argmax(predictions, axis=1)
targetnames = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
#getting the true labels per image 
y_true = test_batches.classes



# Creating classification report 
report = classification_report(y_true, y_pred, target_names=targetnames)

print("\nClassification Report AFTER TRAINING for all 7 calsses :")
print(report)


Classification Report AFTER TRAINING for all 7 calsses :
              precision    recall  f1-score   support

       akiec       0.55      0.26      0.35        23
         bcc       0.55      0.62      0.58        26
         bkl       0.53      0.59      0.56        66
          df       0.10      0.17      0.12         6
         mel       0.22      0.62      0.32        34
          nv       0.96      0.87      0.91       663
        vasc       0.75      0.90      0.82        10

    accuracy                           0.81       828
   macro avg       0.52      0.57      0.52       828
weighted avg       0.87      0.81      0.83       828



#Converting the model to a TfLite model

In [54]:
# Convert Keras model to TF Lite format using quantization.
#Quantization is the process of mapping continuous infinite values to a smaller set of discrete finite values
converter = tf.lite.TFLiteConverter.from_keras_model(model)

converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\JANIDU~1\AppData\Local\Temp\tmpzjs6wx75\assets


In [55]:
# Show model size in MBs.
quantized_model_size = len(tflite_quantized_model) /1024/1024
print(quantized_model_size)

2.593109130859375


In [57]:
# Save the quantized model to files Downloads directory
f = open('SDGP_CNN_Model.tflite', "wb")
f.write(tflite_quantized_model)
f.close()

# Download the skin cancer classification model
#from google.colab import files
#files.download('SDGP_CNN_Model.tflite')

print('SDGP_CNN_Model.tflite has been downloaded')

SDGP_CNN_Model.tflite has been downloaded
